In [9]:
###############################################################################
# This notebook loads the classic excel format for school accreditation data  #
# Current list of featuress:                                                  #
#  * Cleanup of data                                                          #
#  * Process and load back into a clean excel                                 #
###############################################################################
# Core stuff
import os
from pathlib import Path
import re

# Data stuff
import pandas as pd # Data analysis
import numpy as np

# Pretty printing stuff
from IPython.display import display, HTML
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Initial setup
country = 'RMI' # FSM
user = 'ghachey@nuzusys.com'

In [8]:
###############################################################################
# Responses Sheet                                                             #
###############################################################################

# Load a single SOE Assessment workbook (for testing,)
# in particular the sheet with the raw data
cwd = os.getcwd()
filename = os.path.join(cwd, 'data/RMI/school-accreditation-not-yet-in-miemis-2021-22-final.xlsx')

schaccr_dict = pd.read_excel(filename, sheet_name=None, index_col=None, header=0, engine='openpyxl')
print('schaccr_dict')
display(schaccr_dict['Sheet1'])

schaccr_dict


,School,schNo,Date,L1,L2,L3,L4,T1,T2,T3,...,F2,F3,F4,S1,S2,S3,S4,CO1,CO2,Note
0,Airok Elementary School,AIL100,2021-12-31,4,3,2,2,2,2,3,...,3,3,3,2,1,1,1,4,3,Uploaded manually remaining schools with no cl...
1,Enewa Elementary School,AIL102,2021-12-31,3,3,3,2,2,2,2,...,2,3,3,2,2,2,2,2,3,Uploaded manually remaining schools with no cl...
2,Narmij Elementary School,JAL108,2021-12-31,3,3,3,2,2,2,2,...,2,2,1,2,2,2,2,2,2,Uploaded manually remaining schools with no cl...
3,Namu Elementary School,NAU104,2021-12-31,2,2,3,2,2,2,2,...,3,3,3,2,2,2,2,2,2,Uploaded manually remaining schools with no cl...
4,St. Joseph Elementary School,JAL109,2021-12-31,3,2,1,2,3,3,1,...,3,2,2,2,2,2,2,4,3,Uploaded manually remaining schools with no cl...


In [4]:
print(schaccr_dict.keys())

#df_schaccr = pd.concat(
#    [schaccr_dict['Cycle 1-2021'], schaccr_dict['Cycle 1-2020'], schaccr_dict['Cycle 1 - 2019'], 
#     schaccr_dict['Cycle 1-2018'], schaccr_dict['Cycle 1-2017'], schaccr_dict['Initial-2016'], 
#     schaccr_dict['Initial-2015'], schaccr_dict['Initial-2014']
#])
df_schaccr = pd.concat(
    [schaccr_dict['Sheet1']
])

#df_schaccr = df_schaccr

# Put 0 in places where there is no value and change type to int
df_schaccr['CO1'].fillna(0, inplace=True)
df_schaccr['CO2'].fillna(0, inplace=True)
df_schaccr['CO1'] = df_schaccr['CO1'].astype(int)
df_schaccr['CO2'] = df_schaccr['CO2'].astype(int)
print('Types:')
print(df_schaccr.dtypes)

df_schaccr.reset_index(inplace=True, drop=True)

print('School Accreditations:')
display(df_schaccr)

dict_keys(['Sheet1'])
Types:
School            object
schNo             object
Date      datetime64[ns]
L1                 int64
L2                 int64
L3                 int64
L4                 int64
T1                 int64
T2                 int64
T3                 int64
T4                 int64
D1                 int64
D2                 int64
D3                 int64
D4                 int64
N1                 int64
N2                 int64
N3                 int64
N4                 int64
F1                 int64
F2                 int64
F3                 int64
F4                 int64
S1                 int64
S2                 int64
S3                 int64
S4                 int64
CO1                int64
CO2                int64
Note              object
dtype: object
School Accreditations:


,School,schNo,Date,L1,L2,L3,L4,T1,T2,T3,...,F2,F3,F4,S1,S2,S3,S4,CO1,CO2,Note
0,Airok Elementary School,AIL100,2021-12-31,4,3,2,2,2,2,3,...,3,3,3,2,1,1,1,4,3,Uploaded manually remaining schools with no cl...
1,Enewa Elementary School,AIL102,2021-12-31,3,3,3,2,2,2,2,...,2,3,3,2,2,2,2,2,3,Uploaded manually remaining schools with no cl...
2,Narmij Elementary School,JAL108,2021-12-31,3,3,3,2,2,2,2,...,2,2,1,2,2,2,2,2,2,Uploaded manually remaining schools with no cl...
3,Namu Elementary School,NAU104,2021-12-31,2,2,3,2,2,2,2,...,3,3,3,2,2,2,2,2,2,Uploaded manually remaining schools with no cl...
4,St. Joseph Elementary School,JAL109,2021-12-31,3,2,1,2,3,3,1,...,3,2,2,2,2,2,2,4,3,Uploaded manually remaining schools with no cl...


In [5]:
schools_lookup_byname = { # From MIEMIS
'Aerok Elementary School': 'MAL101', 
'Ailuk Elementary School': 'ALU101', 
'Airok Elementary School': 'AIL100', 
'Ajeltake Christian Academy': 'MAJ102', 
'Ajeltake Elementary School': 'MAJ101', 
'Arno Elementary School': 'ARN101', 
'Assumption Elementary School': 'MAJ103', 
'Assumption High School': 'MAJ104', 
'Aur Elementary School': 'AUR101', 
'Bikarej Elementary School': 'ARN102', 
'Buoj Elementary School': 'AIL101', 
'Carlos Elementary School': 'KWA101', 
'Delap Elementary School': 'MAJ105', 
'Delap Kindergarten': 'MAJ106', 
'Delap SDA Elementary School': 'MAJ107', 
'Delap SDA High School': 'MAJ108', 
'Ebadon Elementary School': 'KWA102', 
'Ebeye Calvary Elementary School': 'KWA103', 
'Ebeye Calvary High School': 'KWA104', 
'Ebeye Christian Elementary School': 'KWA105', 
'Ebeye Kinder Elementary School': 'KWA106', 
'Ebeye Public Elementary School': 'KWA108', 
'Ebeye Public Middle School': 'KWA107', 
'Ebeye SDA Elementary School': 'KWA109', 
'Ebeye SDA High School': 'KWA110', 
'Ebon Elementary School': 'EBO101', 
'Ejit Elementary School': 'KIL101', 
'Enejelar Elementary School': 'ALU102', 
'Enejet Elementary School': 'MIL101', 
'Enekoion Elementary School': 'EBO102', 
'Enewa Elementary School': 'AIL102', 
'Enewetak Elementary School': 'ENE101', 
'Enniburr Elementary School': 'KWA111', 
'Enniburr High School': 'KWA119', 
'Father Hacker High School': 'KWA118', 
'GEM Elementary School': 'KWA112', 
'GEM High School': 'KWA113', 
'Imiej Elementary School': 'JAL101', 
'Imroj Elementary School': 'JAL102', 
'Ine Elementary School': 'ARN103', 
'Jabat Elementary School': 'JAB101', 
'Jabnodren Elementary School': 'JAL103', 
'Jabor Elementary School': 'JAL104', 
'Jabro Elementary School': 'KWA115', 
'Jabro High School': 'KWA114', 
'Jah Elementary School': 'AIL103', 
'Jaluit Elementary School': 'JAL105', 
'Jaluit High School': 'JAL106', 
'Jang Elementary School': 'MAL102', 
'Japo Elementary School': 'ARN104', 
'Jebal Elementary School': 'LIK101', 
'Jeh Elementary School': 'AIL104', 
'Jobwan Elementary School': 'AIL105', 
'Katiej Elementary School': 'AIL106', 
'Kaven Elementary School': 'MAL103', 
'Kilange Elementary School': 'ARN105', 
'Kili Elementary School': 'KIL102', 
'Kwajalein Atoll High School': 'KWA116', 
'Lae Elementary School': 'LAE101', 
'Laura Elementary School': 'MAJ109', 
'Laura High School': 'MAJ111', 
'Laura SDA Elementary School': 'MAJ110', 
'Lib Elementary School': 'LIB101', 
'Life Skills Academy': 'MAJ113', 
'Likiep Elementary School': 'LIK102', 
'Loen Elementary School': 'NAU101', 
'Long Island Elementary School': 'MAJ112', 
'Longar Elementary School': 'ARN106', 
'Lukoj Elementary School': 'ARN107', 
'Lukonwod Elementary School': 'MIL102', 
'Mae Elementary School': 'NAU102', 
'Majkin Elementary School': 'NAU103', 
'Majuro Baptist Elementary School': 'MAJ114', 
'Majuro Baptist High School': 'MAJ115', 
'Majuro Coop Elementary School': 'MAJ116', 
'Majuro Coop High School': 'MAJ117', 
'Majuro Middle School': 'MAJ120', 
'Marshall Christians High School': 'MAJ118', 
'Marshall Islands High School': 'MAJ119', 
'Matolen Elementary School': 'ARN108', 
'Mejatto Elementary School': 'RON101', 
'Mejel Elementary School': 'AIL107', 
'Mejit Elementary School': 'MEJ101', 
'Mejrirok Elementary School': 'JAL107', 
'Melang Elementary School': 'LIK103', 
'Mili Elementary School': 'MIL103', 
'Nallo Elementary School': 'MIL104', 
'Namdrik Elementary School': 'NAM101', 
'Namu Elementary School': 'NAU104', 
'Narmij Elementary School': 'JAL108', 
'North Delap Elementary School': 'MAJ126', 
'Northern Islands High School': 'WTH101', 
'Ollet Elementary School': 'MAL104', 
'Queen of Peace Elementary School': 'KWA117', 
'Rairok Elementary School': 'MAJ121', 
'Rita Christian Elementary School': 'MAJ123', 
'Rita Christian High School': 'MAJ124', 
'Rita Elementary School': 'MAJ122', 
'RonRon Protestant Elementary School': 'MAJ125', 
'St. Joseph Elementary School': 'JAL109', 
'Tarawa Elementary School': 'MAL105', 
'Tinak Elementary School': 'ARN109', 
'Tobal Elementary School': 'AUR102', 
'Toka Elementary School': 'EBO103', 
'Tokewa Elementary School': 'MIL105', 
'Tutu Elementary School': 'ARN110', 
'Ujae Elementary School': 'UJA101', 
'Ulien Elementary School': 'ARN111', 
'Utrik Elementary School': 'UTR101', 
'Wodmej Elementary School': 'WTH102', 
'Woja Elementary School (Ailinglaplap)': 'AIL108', 
'Woja Elementary School (Majuro)': 'MAJ127', 
'Wotho Elementary School': 'WOT101', 
'Wotje Elementary School': 'WTH103', # That did not map to MIEMIS precisely
'Bouj Elementary School': 'AIL101',
'Ebeye Kindergarten': 'KWA106',
'Jepal Elementary School': 'LIK101',
'Delap Elementary Sch.': 'MAJ105',
'Assumption Elementary Sch.': 'MAJ103',
'Coop Elementary School': 'MAJ116',
'Coop High School': 'MAJ117',
'Bouj Elementary School': 'AIL101',
'Marshall Christian High School': 'MAJ118',
'Ebeye Kindergarten': 'KWA106',
'Bouj Elementary School': 'AIL101',
'Jepal Elementary School': 'LIK101',
'Enejet Elementary School ': 'MIL101',   
'Wodmej Elementary School Primary': 'WTH102',
'Tobal Elementary School ': 'AUR102', 
'Life Skill Academy': 'MAJ113', 
'DUD Kinder': 'MAJ126', # Old merged to North Delap Elementary Schools
'Uliga Elementary School': 'MAJ126' # Old merged to North Delap Elementary Schools
}

# Add schNo column
s_school_ids = df_schaccr['School'].map(schools_lookup_byname)
s_school_ids
df_schaccr['schNo'] = s_school_ids

# Check if any schools have not been mapped
print('Remaining rows without a schNo:')
display(df_schaccr[df_schaccr['schNo'].isna()])

# For now remove Marshall Christian Elementary School as it is not counted in their
# report nor is there a school in MIEMIS for it
df_schaccr = df_schaccr[df_schaccr['School'] != 'Marshall Christian Elementary School']

# Re-order columns and keep only those needed
df_schaccr2 = df_schaccr.copy()
df_schaccr2 = df_schaccr2[['schNo',
 'Date',
 'L1',
 'L2',
 'L3',
 'L4',
 'T1',
 'T2',
 'T3',
 'T4',
 'D1',
 'D2',
 'D3',
 'D4',
 'N1',
 'N2',
 'N3',
 'N4',
 'F1',
 'F2',
 'F3',
 'F4',
 'S1',
 'S2',
 'S3',
 'S4',
 'CO1',
 'CO2',
 'Note']]
df_schaccr2

print('School Accreditation DataFrame')
display(df_schaccr2)

Remaining rows without a schNo:


,School,schNo,Date,L1,L2,L3,L4,T1,T2,T3,...,F2,F3,F4,S1,S2,S3,S4,CO1,CO2,Note


School Accreditation DataFrame


,schNo,Date,L1,L2,L3,L4,T1,T2,T3,T4,...,F2,F3,F4,S1,S2,S3,S4,CO1,CO2,Note
0,AIL100,2021-12-31,4,3,2,2,2,2,3,2,...,3,3,3,2,1,1,1,4,3,Uploaded manually remaining schools with no cl...
1,AIL102,2021-12-31,3,3,3,2,2,2,2,2,...,2,3,3,2,2,2,2,2,3,Uploaded manually remaining schools with no cl...
2,JAL108,2021-12-31,3,3,3,2,2,2,2,2,...,2,2,1,2,2,2,2,2,2,Uploaded manually remaining schools with no cl...
3,NAU104,2021-12-31,2,2,3,2,2,2,2,2,...,3,3,3,2,2,2,2,2,2,Uploaded manually remaining schools with no cl...
4,JAL109,2021-12-31,3,2,1,2,3,3,1,2,...,3,2,2,2,2,2,2,4,3,Uploaded manually remaining schools with no cl...


In [12]:
# Generate the SQL for the data to be inserted in a transaction
def sql_insert_statement_from_dataframe(source, target):
    sql_texts = []
    sql_texts_sp = []
    for index, row in source.iterrows():       
        sql_texts.append('INSERT INTO '+target+' ('+ str(', '.join(source.columns))+ ') VALUES '+ str(tuple(row.values)))
        sql_texts_sp.append('EXEC [dbo].[insertSAfromParams_RMI] @schNo = N\''+str(row['schNo'])+'\', @Date = \''+str(row['Date'])+'\', @L1 = '+str(row['L1'])+', @L2 = '+str(row['L2'])+', @L3 = '+str(row['L3'])+', @L4 = '+str(row['L4'])+', @T1 = '+str(row['T1'])+', @T2 = '+str(row['T2'])+', @T3 = '+str(row['T3'])+', @T4 = '+str(row['T4'])+', @D1 = '+str(row['D1'])+', @D2 = '+str(row['D2'])+', @D3 = '+str(row['D3'])+', @D4 = '+str(row['D4'])+', @N1 = '+str(row['N1'])+', @N2 = '+str(row['N2'])+', @N3 = '+str(row['N3'])+', @N4 = '+str(row['N4'])+', @F1 = '+str(row['F1'])+', @F2 = '+str(row['F2'])+', @F3 = '+str(row['F3'])+', @F4 = '+str(row['F4'])+', @S1 = '+str(row['S1'])+', @S2 = '+str(row['S2'])+', @S3 = '+str(row['S3'])+', @S4 = '+str(row['S4'])+', @CO1 = '+str(row['CO1'])+', @CO2 = '+str(row['CO2'])+', @User = \''+user+'\''', @note = \''+str(row['Note'])+'\';')

    return sql_texts_sp

db_statements = sql_insert_statement_from_dataframe(df_schaccr2, '[dbo].[SchoolInspections]')

filename = os.path.join(cwd, 'data/RMI/school-accredition-for-loading-to-sql-db-2021-22.sql')
with open(filename, 'w') as f:
    f.write("\n".join(db_statements))

In [ ]:
# Write various DataFrame into Excel to examine (testing)
filename = os.path.join(cwd, 'data/RMI/school-accredition-for-loading-to-sql-db.xlsx')
with pd.ExcelWriter(filename) as writer:
    # add DataFrames you want to write to Excel here
    df_schaccr.to_excel(writer, index=False, sheet_name='Sheet1', engine='openpyxl')
    df_schaccr2.to_excel(writer, index=False, sheet_name='Sheet2', engine='openpyxl')
    #df_students_results_and_enrol.to_excel(writer, index=False, sheet_name='Sheet2', engine='openpyxl')
    #df_onlinesba.to_excel(writer, index=False, sheet_name='Sheet4', engine='openpyxl')